# Neural search for question answering
Mateusz Wojtulewicz

## Setup
I'm setting up Elasticsearch to work in Google Colab, installing Haystack fork with Polish QA models support as well as installing useful libraries.

In [1]:
from IPython.display import clear_output

!pip install elasticsearch 
!pip install elasticsearch-async
!pip install git+https://github.com/apohllo/haystack.git@use-auto-tokenizer-by-default
!pip install faiss-gpu
!pip install sacremoses

clear_output()
print("Done.")

Done.


In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.4.3-linux-x86_64.tar.gz
tar -xzf elasticsearch-8.4.3-linux-x86_64.tar.gz

sudo chown -R daemon:daemon elasticsearch-8.4.3/
umount /sys/fs/cgroup
apt install cgroup-tools

In [3]:
! elasticsearch-8.4.3/bin/elasticsearch-plugin install pl.allegro.tech.elasticsearch.plugin:elasticsearch-analysis-morfologik:8.4.3

-> Installing pl.allegro.tech.elasticsearch.plugin:elasticsearch-analysis-morfologik:8.4.3
-> Downloading pl.allegro.tech.elasticsearch.plugin:elasticsearch-analysis-morfologik:8.4.3 from maven central
[=================================================] 100%   
-> Installed analysis-morfologik
-> Please restart Elasticsearch to activate any plugins installed


In [6]:
%%bash --bg

sudo -H -u daemon elasticsearch-8.4.3/bin/elasticsearch

In [7]:
# wait ~20s for elastic search to start
# then stop it

! pkill -f elasticsearch

# IMPORTANT
# disable security features
# set xpack.security.enable: false
# in elasticsearch-8.4.3/config/elasticsearch.yml

In [75]:
%%bash --bg

sudo -H -u daemon elasticsearch-8.4.3/bin/elasticsearch

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
!wget https://apohllo.pl/text/ustawy.tar.gz
!mkdir ustawy
!tar -xf ustawy.tar.gz -C ustawy

clear_output()
print("Done.")

Done.


## Lab

In [33]:
import os
import json
import tqdm
import time
import requests
import pandas as pd
import numpy as np

from pathlib import Path

from haystack.document_stores import ElasticsearchDocumentStore, FAISSDocumentStore
from haystack.nodes import BM25Retriever, TransformersReader, DensePassageRetriever, BaseRetriever
from haystack.pipelines import ExtractiveQAPipeline

In [12]:
ACTS_DIR = Path("ustawy")
ES_URL = "http://localhost:9200"
INDEX = "passages"

### Pre-processing documents
I'm splitting documents from the set of Polish bills into individual articles by using a simple heuristic that searches for `Art.` at the beginning of a line. Every individual article with assigned identifier is stored in `passages` dictionary.

In [13]:
passages = []

for act in tqdm.tqdm(ACTS_DIR.iterdir()):
    act_id = act.stem
    with act.open() as f:
        lines = f.readlines()
    
    passage_id = None
    contents = []

    for line in lines:
        line = line.strip()
        
        if line.startswith("Art. "):
            if passage_id is not None:
                passage_dict = {
                    "content": " ".join(contents),
                    "meta": {
                        "id": f"{act_id}_{passage_id}"
                    }
                }
                passages.append(passage_dict)
            
            passage_id = line[5:-1]
            contents = []
        
        else:
            if passage_id is not None:
                contents.append(line)
    
    if passage_id is not None:
        passage_dict = {
            "content": " ".join(contents),
            "meta": {
                "id": f"{act_id}_{passage_id}"
            }
        }
        passages.append(passage_dict)

1179it [00:00, 1996.44it/s]


In [14]:
print(f"There are {len(passages)} passages.")

There are 24752 passages.


### Configuring document stores
I'm configuring two document stores: one based on ElasticSearch (with polish language analyzer) and one based on Faiss (with proper polish encoders).

#### Elasticsearch
I'm creating a `passages` index, with text context field that uses polish analyzer designed in previous lab.

In [15]:
# pretty printing ES responses
def my_pprint(response):
    if isinstance(response, requests.Response):
        response = response.json()
    print(json.dumps(response, indent=4, ensure_ascii=False))

In [ ]:
response = requests.put(
    url=f"{ES_URL}/{INDEX}",
    json={
        "settings": {
            "analysis": {
                "analyzer": {
                    "polish-law-analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "synonym-filter",
                            "morfologik_stem",
                            "lowercase"
                        ]
                    }
                },
                "filter": {
                    "synonym-filter": {
                        "type": "synonym",
                        "synonyms": [
                            "kpk => kodeks postępowania karnego",
                            "kpc => kodeks postępowania cywilnego",
                            "kk => kodeks karny",
                            "kc => kodeks cywilny"
                        ]
                    }
                }
            }
        }
    }
)

In [17]:
response = requests.put(
    url=f"{ES_URL}/{INDEX}/_mapping",
    json={
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "polish-law-analyzer"
            }
        }
    }
)

my_pprint(response)

{
    "acknowledged": true
}


In [18]:
docstore_es = ElasticsearchDocumentStore(username="", password="", index=INDEX)
retriever_es = BM25Retriever(document_store=docstore_es)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


#### Faiss
I'm using encoder models for polish QA.

In [19]:
docstore_faiss = FAISSDocumentStore()
retriever_faiss = DensePassageRetriever(
    document_store=docstore_faiss,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context",
    use_gpu=True,
)

INFO:haystack.modeling.utils:Using devices: CUDA:0
INFO:haystack.modeling.utils:Number of GPUs: 1


Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/886k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:LOADING MODEL
INFO:haystack.modeling.model.language_model:=============
INFO:haystack.modeling.model.language_model:Could not find enelpol/czywiesz-context locally.
INFO:haystack.modeling.model.language_model:Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Loaded enelpol/czywiesz-context


### Loading the passages to document stores

#### Elasticsearch

In [20]:
start_time = time.time()
docstore_es.write_documents(passages)
print(f"{time.time() - start_time:.3f}s")

61.807s


#### Faiss
In Faiss I'm not only writing passages to document store but also updating embeddings using encoder models.

In [21]:
start_time = time.time()
docstore_faiss.write_documents(passages)
docstore_faiss.update_embeddings(retriever_faiss)
print(f"{time.time() - start_time:.3f}s")

Writing Documents:   0%|          | 0/24752 [00:00<?, ?it/s]

INFO:haystack.document_stores.faiss:Updating embeddings for 24223 docs...


Updating Embedding:   0%|          | 0/24223 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/4224 [00:00<?, ? Docs/s]

2063.590s


### Evaluating performace of both document stores
I'm using polish questions from `simple-legal-questions-pl` dataset to calculate `Pr@1`, `Rc@1`, `Pr@3` and `Rc@3` metrics. 

In [27]:
! unzip drive/MyDrive/studia/9semestr/nlp/simple-legal-questions-pl.zip

clear_output()
print("Done.")

Done.


In [38]:
df = pd.read_csv("simple-legal-questions-pl/questions.csv")
df = df.loc[df.has_answer == True]
df["passage_id"] = df.apply(lambda row: f"{row.year}_{row.position}_{row.art}", axis=1)

df

,passage_id,question,passage,has_answer,year,position,art
0,1997_553_345,"Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega ka...","Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełoż...",True,1997,553,345
1,2004_177_21,Z ilu osób składa się komisja przetargowa?,Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zam...,True,2004,177,21
2,1996_465_111,Do jakiej wysokości za zobowiązania spółki odpowiada komandytariusz?,Art. 111. Komandytariusz odpowiada za zobowiązania spółki wobec jej wierzy...,True,1996,465,111
3,1994_591_35,"Kiedy ustala się wartość majątku obrotowego, który stracił swoją przydatność?","Art. 35. 1. Wartość rzeczowych składników majątku obrotowego, które utr...",True,1994,591,35
4,2001_1441_74,"Jakiej karze podlega armator, który wykonuje rybołówstwo morskie w polskich...","Art. 74. 1. Armator, który wykonuje rybołówstwo morskie w polskich obszara...",True,2001,1441,74
...,...,...,...,...,...,...,...
1470,1995_479_29,Jakim przepisom podlegają przychody kościelnych osób prawnych?,1. Majątek i przychody kościelnych osób prawnych podlegają ogólnym przepisom...,True,1995,479,29
1471,1995_482_27,Jakim przepisom podlegają przychody kościelnych osób prawnych?,1. Majątek i przychody kościelnych osób prawnych podlegają ogólnym przepisom...,True,1995,482,27
1472,1997_554_19,Jakim przepisom podlegają przychody kościelnych osób prawnych?,1. Majątek i przychody kościelnych osób prawnych podlegają ogólnym przepisom...,True,1997,554,19
1473,1995_481_28,Jakim przepisom podlegają przychody kościelnych osób prawnych?,1. Majątek i przychody Kościoła oraz jego osób prawnych podlegają ogólnym pr...,True,1995,481,28


In [40]:
def precision(preds: set, relevant: set) -> float:
    return len(preds.intersection(relevant)) / len(preds)

def recall(preds: set, relevant: set) -> float:
    return len(preds.intersection(relevant)) / len(relevant)

In [66]:
def evaluate(
    retriever: BaseRetriever,
    data: pd.DataFrame, 
) -> dict:
    questions = data.question.unique()

    pr1 = np.zeros_like(questions)
    pr3 = np.zeros_like(questions)
    rc1 = np.zeros_like(questions)
    rc3 = np.zeros_like(questions)

    for i, question in enumerate(questions):
        data_relevant = data.loc[data.question == question]
        relevant_ids = set(data_relevant.passage_id)

        preds = retriever.retrieve(question, top_k=3)
        preds_ids = [d.meta["id"] for d in preds]

        preds_top1 = set([preds_ids[0]])
        preds_top3 = set(preds_ids)

        pr1[i] = precision(preds_top1, relevant_ids)
        pr3[i] = precision(preds_top3, relevant_ids)

        rc1[i] = recall(preds_top1, relevant_ids)
        rc3[i] = recall(preds_top3, relevant_ids)

    # print last evaluation details
    print(f"Last evaluation details:")
    print(f"Question: {question}")
    print("Relevant answers:")
    my_pprint(data_relevant.passage.tolist())
    print("Predicted answers:")
    my_pprint([d.content for d in preds])

    metrics = {
        "Pr@1": pr1.mean(),
        "Pr@3": pr3.mean(),
        "Rc@1": rc1.mean(),
        "Rc@3": rc3.mean()
    }

    print(f"Evaluated metrics:")
    my_pprint(metrics)

    return metrics

#### Elasticsearch

In [82]:
start_time = time.time()
metrics = evaluate(retriever=retriever_es, data=df)
print(f"Evaluation time: {time.time() - start_time:.3f}s")

Last evaluation details:
Question: Według jakiego prawa wyraz "kosmetyki" zastępuje się wyrazami "produkty kosmetyczne"?
Relevant answers:
[
    "z dnia 19 marca 2004 r. – Prawo celne (Dz. U. z 2018 r. poz. 167, 1544, 1669 i 1697) w art. 31 w ust. 5 wyraz „kosmetyki” zastępuje się wyrazami „produkty kosmetyczne”. "
]
Predicted answers:
[
    "W ustawie z dnia 30 marca 2001 r. o kosmetykach (Dz.U. Nr 42, poz. 473, z późn. zm.[2] wprowadza się następujące zmiany: 1) w art. 3: a) pkt 1 otrzymuje brzmienie: \"1) producent - przedsiębiorcę, który wytwarza i wprowadza kosmetyk do obrotu albo który wprowadza kosmetyk do obrotu, a także jego przedstawiciela oraz każdą osobę, która występuje jako wytwórca, umieszczając na produkcie lub dołączając do niego swoją firmę, znak towarowy lub inne odróżniające oznaczenie; za producenta uważa się także importera,\", b) w pkt 10 kropkę zastępuje się przecinkiem i dodaje się pkt 11 w brzmieniu: \"11) prototyp kosmetyku - pierwszy model lub projekt, niepr

#### Faiss

In [83]:
start_time = time.time()
metrics = evaluate(retriever=retriever_faiss, data=df)
print(f"Evaluation time: {time.time() - start_time:.3f}s")

Last evaluation details:
Question: Według jakiego prawa wyraz "kosmetyki" zastępuje się wyrazami "produkty kosmetyczne"?
Relevant answers:
[
    "z dnia 19 marca 2004 r. – Prawo celne (Dz. U. z 2018 r. poz. 167, 1544, 1669 i 1697) w art. 31 w ust. 5 wyraz „kosmetyki” zastępuje się wyrazami „produkty kosmetyczne”. "
]
Predicted answers:
[
    "Użyte w ustawie określenia oznaczają: 1) producent - przedsiębiorcę, który wytwarza, wprowadza do obrotu, a także jego przedstawiciela oraz każdą osobę, która występuje jako wytwórca, umieszczając na produkcie bądź do niego dołączając swoje nazwisko, nazwę, znak towarowy bądź inne odróżniające oznaczenie; za producenta uważa się również importera oraz każdego, kto prowadząc działalność gospodarczą może wpływać na bezpieczeństwo kosmetyku, 2) wprowadzenie do obrotu - przekazanie kosmetyku przez producenta po raz pierwszy w kraju: użytkownikowi, konsumentowi bądź przedsiębiorcy uczestniczącemu w obrocie handlowym, 3) składnik kosmetyku - substancję,

### Answering questions

#### 1. Which of the document stores performs better? Take into account the different metrics enumerated in the previous point.

Elasticsearch performes better than Faiss considering both the accuracy metrics and computational time. 

Compared to Faiss, ES obtained more than two times better precision and recall scores in top-1 mode (`0.71` vs `0.31` in precision, `0.60` vs `0.25` in recall), and a little less than two times improvement in top-3 mode (`0.32` vs `0.19` in precision, `0.75` vs `0.44` in recall). 


#### 2. Which of the document stores is faster?

Elasticsearch is much faster considering both the documents writing time (30x faster) and inference time (4x faster). This is caused by the fact that Faiss need to generate neural network outputs for both operations, while Elasticsearch does not.


#### 3. Try to determine the other pros and cons of using sparse and dense document retrieval models.

Sparse retrieval mode appears to be better suited for questions that are well defined, with answers appearing directly in the context. Dense vector representation on the other hand, might perform better for questions more general and vage.

Sparse retrieval might have troubles when presented with words out of their dictionary, while dense mode can learn vector embedding for any word.

Dense mode needs a training phase for the encoder model. It can be less of a problem in present times when pretrained model for any task can be fined and then fine-tuned for certain usage. For comparison, sparse mode does not need any training but the words dictionary has to be build for it. This might be a bigger problem because when new words appear it has to be rebuild not only fine-tuned.
